In [3]:
from lxml import html
import requests

In [2]:
# Download CSV's

user_agent = 'Mozilla/5.0 (X11; Linux x86_64; rv:68.0) Gecko/20100101 Firefox/68.0'
headers = {'User-Agent': user_agent}
url = 'https://www.userbenchmark.com/page/developer'

response = requests.get(url, headers=headers)

page = html.fromstring(response.content)

csvList = page.xpath('//div[@class="list-group"]/a/@href')

for url in csvList:
    urlcomponents = url.split('/')
    filename = urlcomponents[-1]
    with open(filename,'wb') as outputCSV:
        response = requests.get(url, headers=headers)
        outputCSV.write(response.content)

In [11]:
# Setup Browser

from selenium import webdriver
from selenium.webdriver.chrome import service

webdriver_service = service.Service('operadriver.exe')
webdriver_service.start()

driver = webdriver.Remote(webdriver_service.service_url,webdriver.DesiredCapabilities.OPERA)

driver.switch_to_window(driver.window_handles[0])

In [15]:
import csv
import time
import pickle
import os
import random


In [16]:
# Scrape Website and Save Data Locally

with open('CPU_UserBenchmarks.csv', newline='') as inputFile:
    reader = csv.DictReader(inputFile)
    for row in reader:
        filename = row["URL"].split('/')[-1]
        if os.path.exists(f'cpu\{filename}.response'):
            continue
        if "We need to ensure that you are not a robot." in driver.page_source:
            print('robot!')
            break
        driver.get(row['URL'])
        print(row['URL'])
        pickle.dump(driver.page_source, open(f'cpu\{filename}.response','wb'))
        print(filename)

In [149]:
def getXPathValue(xpath, replace=False, removeLast=None, removeFirst=None, number=True):
    global page
    try:
        data = page.xpath(xpath)[0]
    except Exception as e:
        print(e, 'one')
        return
    if replace != False:
        data = data.replace(replace[0],replace[1])
    if not removeLast is None:
        data = data[:removeLast]
    if not removeFirst is None and len(data) > 6:
        data = data[removeFirst:]
    if number:
        return int(data)
    return data

In [161]:
# Data we Want

info = {
    'Effective Speed':{'xpath':'//span[text()="Effective Speed "]/span[1]/text()','replace':False,'removeLast':-2},
    'Average Score':{'xpath':'//span[text()="Average Score "]/span[1]/text()','replace':False,'removeLast':-2},
    'Overclocked Score':{'xpath':'//span[text()="Overclocked Score "]/span[1]/text()','replace':False,'removeLast':-2},
    'Value & Sentiment':{'xpath':'//span[text()="Value & Sentiment "]/span[1]/text()','replace':(',',''),'removeLast':-1},
    # 'Nice To Haves':{'xpath':'//span[text()="Nice To Haves "]/span[1]/text()','replace':False,'removeLast':-2},
    'Code Name':{'xpath':'//td[text()="Code Name"]/../td/span/text()','replace':False,'removeLast':None,'number':False},
    'Launch Date':{'xpath':'//td[text()="Launch Date"]/../td/span/text()','replace':False,'removeLast':None,'number':False},
    'Processor Number':{'xpath':'//td[text()="Processor Number"]/../td/span/text()','replace':False,'removeLast':None,'number':False},
    '# of Cores':{'xpath':'//td[text()="# of Cores"]/../td/span/text()','replace':False,'removeLast':None},
    '# of Threads':{'xpath':'//td[text()="# of Threads"]/../td/span/text()','replace':False,'removeLast':None},
    'Max TDP':{'xpath':'//td[text()="Max TDP"]/../td/span/text()','replace':False,'removeLast':-2},
    'Price':{'xpath':'//span[@id="stickypriceid"]/a/span/text()','replace':False,'removeFirst':7},
    'Sockets Supported':{'xpath':'//td[text()="Sockets Supported"]/../td/span/text()','number':False}
}

In [165]:
# Process Scraped Files for Data we Want

import csv
import time
import pickle

count = 0

with open('CPU_UserBenchmarks.csv', newline='') as inputFile:
    reader = csv.DictReader(inputFile)
    with open('cpu.csv', 'w', newline='') as outputFile:
        first = True
        for row in reader:
            if first:
                fieldnames = reader._fieldnames
                fieldnames.extend(info.keys())
                writer = csv.DictWriter(outputFile, fieldnames)
                writer.writeheader()
                first = False
            filename = row["URL"].split('/')[-1]
            try:
                meh = pickle.load(open(f'cpu\{filename}.response','rb'))
                if type(meh) is str:
                    page = html.fromstring(meh)
                else:
                    page = html.fromstring(meh.content)
            except Exception as e:
                print('failed',filename, e)
                break
            print(filename)
            for key in info:
                try:
                    row[key] = getXPathValue(**info[key])
                except Exception as e:
                    print(e, key)
                    continue
            writer.writerow(row)
            count += 1

st index out of range one
--------IntelR-PentiumR-M-processor-186GHz
invalid literal for int() with base 10: '-1,52' Average Score
invalid literal for int() with base 10: '-1,44' Overclocked Score
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
AMD-Athlontm-64-X2-Dual-Core-Processor-TK-55
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
IntelR-CeleronR-N4000-CPU---110GHz
list index out of range one
AMD-E-350D-APU-with-Radeontm-HD-Graphics
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index out of range one
list index

In [ ]:
driver.quit()